In [1]:
import numpy as np
import sys, os
import time
import glob
from registration_3d import *
from cartesian import *
from collections import namedtuple
from operator import itemgetter
from pprint import pformat

In [2]:
def read_mesh( data_dir , filename ):
# get the coordinate of points
# get the vertices of each triangle
    mesh = glob.glob( data_dir + filename )
    mesh_file = open( mesh[0] , "r")
    mesh_lines = mesh_file.read().splitlines()
    mesh_data = []
    
    for num in range(len(mesh_lines)):
        mesh_data.append( mesh_lines[num].split(' ') )
        for i in range(len( mesh_data[-1])):
            mesh_data[-1][i] = mesh_data[-1][i].strip() # to remove space and tabs
    Np = int(mesh_data[0][0])
    mesh_data = mesh_data[1:]
    
    mesh_points = np.asarray(mesh_data[0:Np]).astype(float)
    mesh_data = mesh_data[Np:]
    
    Ntriangles = int(mesh_data[0][0])
    mesh_data = mesh_data[1:]
    mesh_vertices = np.asarray(mesh_data[0:Ntriangles]).astype(int)
    mesh_data = mesh_data[Ntriangles:]
    
    return Np,mesh_points,Ntriangles,mesh_vertices

In [3]:
def read_body( data_dir , filename ):

# get the markers' positions and tip of a body
    body = glob.glob( data_dir + filename )
    body_file = open( body[0] , "r")
    body_lines = body_file.read().splitlines()
    body_data = []
    
    for num in range(len(body_lines)):
        body_data.append( body_lines[num].split() )
        for i in range(len( body_data[-1])):
            body_data[-1][i] = body_data[-1][i].strip() # to remove space and tabs
    Np = int(body_data[0][0])
    body_data = body_data[1:]
    
    body_points = np.asarray(body_data[0:Np]).astype(float)
    body_data = body_data[Np:]
    
    tip = body_data[0]
    tip = np.array(tip).astype(float)
    tip = tip.reshape(1,-1)
    return Np,body_points,tip


In [4]:
def read_sample( data_dir , filename , N_Ap , N_Bp ):
# get the markers' positions and tip of a body
# N_Ap is the number of marker in A
# N_Bp is the number of marker in B
    sample = glob.glob( data_dir + filename )
    sample_file = open( sample[0] , "r")
    sample_lines = sample_file.read().splitlines()
    sample_data = []
    
    for num in range(len(sample_lines)):
        sample_data.append( sample_lines[num].split(',') )
        for i in range(len( sample_data[-1])):
            sample_data[-1][i] = sample_data[-1][i].strip() # to remove space and tabs
    Np = int(sample_data[0][0])
    N_sample = int(sample_data[0][1])
    N_Dp = Np - N_Ap - N_Bp
    sample_data = sample_data[1:]
    sample_data = np.array(sample_data).astype(float)
    A_frames=[]
    B_frames=[]
    
    for i in range(N_sample):
        A_sample_points = np.asarray( sample_data[ i*Np : i*Np + N_Ap ] ).astype( float )
        A_frames.append( A_sample_points )
        
        B_sample_points = np.asarray( sample_data[ i*Np + N_Ap :  i*Np + N_Ap + N_Bp] ).astype( float )
        B_frames.append( B_sample_points )

    A_frames = np.array(A_frames).astype(float)
    B_frames = np.array(B_frames).astype(float)
        
    return A_frames , B_frames

In [5]:
def get_d_k(A_frames, A_points, A_tip, B_frames, B_points, B_tip):
    
    d_k = []
    for i in range(len(A_frames)):
        F_AK = registration_3d(A_points , A_frames[i])
        F_BK = registration_3d(B_points , B_frames[i])
        # F_AK is the transformation from A tool frame to optical frame
        # F_AK is the transformation from B tool frame to optical frame
        d_k_i = points_transform( Fi(F_BK) @ F_AK , A_tip)
        d_k.append(d_k_i)
    d_k = np.array(d_k).astype(float)
    d_k = d_k.reshape(-1,3)
    return d_k

In [6]:
def ProjectOnSegment(c,p,q):
# c is the point we want to find corresponding closest point
# p q are the start and the and of a segment

#according to https://ciis.lcsr.jhu.edu/lib/exe/fetch.php?media=courses:455-655:lectures:finding_point-pairs.pdf
# page 7
    c = np.array(c).astype(float)
    p = np.array(p).astype(float)
    q = np.array(q).astype(float)
    #print(np.dot( c-p , q-p ))
    #print(np.dot( q-p ,q-p ))
    Lambda = np.dot( c-p , q-p ) / np.dot( q-p ,q-p )
    Lambda = max(0, min(Lambda, 1))
    c_star = p + Lambda*(q-p)
    return c_star

In [7]:
def FindClosestPoint( a, p, q, r ):
    
    A = np.zeros((3,2),dtype = float )
    
    for i in range(3):
        A[i][0] = q[i] - p[i]
        A[i][1] = r[i] - p[i]
        
    B = a - p
    B = B.reshape(-1,1)
    X = np.linalg.lstsq(A, B, rcond=None)[0]
    
    Lambda = X[0]
    Miu = X[1]

    a = p + Lambda * (q - p) + Miu * (r - p)
    P_closest = np.zeros(3)

    if Lambda >= 0 and Miu>=0 and Lambda+Miu<=1:
        P_closest = a
    elif Lambda < 0:
        P_closest = ProjectOnSegment(a, r, p)
    elif Miu < 0:
        P_closest = ProjectOnSegment(a, p, q)
    elif Lambda + Miu > 1:
        P_closest = ProjectOnSegment(a, q, r)

    return P_closest


In [8]:
def point_to_triangle(s_k_i , j):
    p = mesh_points[mesh_vertices[j][0]] 
    q = mesh_points[mesh_vertices[j][1]]
    r = mesh_points[mesh_vertices[j][2]] 
            
    tmp = FindClosestPoint( s_k_i, p, q, r )
    
    return tmp

In [9]:
def bruteforce_matching(s_k , Np , mesh_points , Ntriangles , mesh_vertices ):
    Ns = s_k.shape[0]
    
    closest_p = []
    for i in range(Ns):
        min_dist = np.finfo(np.float32).max
        P_closest = []
        for j in range(Ntriangles):

            tmp = point_to_triangle(s_k[i] , j)

            dist = np.linalg.norm(s_k[i] - tmp)
            if min_dist > dist:
                min_dist = dist
                P_closest = tmp
        closest_p.append(P_closest)
                
    return closest_p

In [10]:
def get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices):
    
    rectangles = np.zeros( (Ntriangles,7) ,dtype=float)
    
    for i in range(Ntriangles):
        p = mesh_points[mesh_vertices[i][0]] 
        q = mesh_points[mesh_vertices[i][1]]
        r = mesh_points[mesh_vertices[i][2]]
        rectangles[i,6] = i
        for j in range(3):
            rectangles[i,j] = min( p[j],q[j],r[j])
        for j in range(3):
            rectangles[i,j+3] = max( p[j],q[j],r[j])
        #print(rectangle[i,0:3] - rectangle[i,3:6])
        
    return rectangles

In [11]:
class KD_tree():
    def __init__(self, Np , mesh_points , Ntriangles , mesh_vertices , rectangles):
        
        self.Np = Np
        self.mesh_points = mesh_points
        self.mesh_vertices = mesh_vertices
        self.Ntriangles =  Ntriangles
        self.rectangles = rectangles
        self.num = np.zeros(Ntriangles+10).astype(int)
        #self.D = mesh_points.shape[1]
        self.D = 3
        
        self.ls = np.zeros( Ntriangles+10 )
        self.rs = np.zeros( Ntriangles+10 )
        
        #self.points = rectangles
        self.Rectangle = np.zeros( (Ntriangles+10,6) )
        
        self.tree = self.build(rectangles.tolist(), 1 , Ntriangles , depth =0 )
        self.nearest_point = []
        self.nearest_triangle = []
        self.nearest_dist = 0
        
    def pushup(self, root):
        
        ls = self.ls[root]
        rs = self.rs[root]
        if(ls!=0):
            for i in range(3):
                self.Rectangle[root,i] = min(self.Rectangle[root,i],self.Rectangle[int(ls),i])
                self.Rectangle[root,i+3] = max(self.Rectangle[root,i+3],self.Rectangle[int(ls),i+3]) 
        if(rs!=0):
            for i in range(3):
                self.Rectangle[root,i] = min(self.Rectangle[root,i],self.Rectangle[int(rs),i])
                self.Rectangle[root,i+3] = max(self.Rectangle[root,i+3],self.Rectangle[int(rs),i+3]) 
    
    def point_to_cube(self, start_point, root):
        dis = np.zeros(3)
        for i in range(self.D):
            if(start_point[i] < self.Rectangle[root,i]):
                dis[i] = self.Rectangle[root,i] - start_point[i]
            if(start_point[i] > self.Rectangle[root,i+3]):
                dis[i] = start_point[i] - self.Rectangle[root,i+3]
        dist = np.linalg.norm(dis)
        return dist
    
    def find(self, start_point , left,right,depth):
        if(left>right):
            return 
        middle = ((left + right) // 2) 
        
        dist = self.point_to_cube(start_point , middle)
        if(dist > self.nearest_dist):
            return
        
        tmp = point_to_triangle(start_point , self.num[middle])
        dist = np.linalg.norm(start_point - tmp)
        
        if( dist < self.nearest_dist):
            self.nearest_dist = dist
            self.nearest_point = tmp
            self.nearest_triangle = self.num[middle]
        
        self.find( start_point , left , middle-1 ,depth)
        self.find( start_point , middle+1 , right,depth)
        
    def FindClosestPoint(self, start_point ):
        
        self.nearest_dist =  np.finfo(np.float32).max
        self.nearest_triangle =  -1
        self.find( start_point , 1 , self.Ntriangles , depth=0 ) 
        
        return self.nearest_point , self.nearest_triangle
        
    def build( self, points, left,right,depth ):
        if(left>right):
            return 0
        
        axis = depth % self.D

        points.sort(key=itemgetter(axis))
        middle = ((left + right) // 2) 
        #print("points: ",len(points))
        #print("middle: ",middle)
        self.Rectangle[middle] = np.array(points[ middle - left ][0:6]).astype(float)
        self.num[middle] =  points[middle - left ][6]
        
        self.ls[ middle ] = self.build(points[:middle- left] ,left , middle-1 , depth+1 )
        self.rs[ middle ] = self.build(points[middle-left+1:]   ,middle+1, right , depth+1 )
        self.pushup(middle)
        
        return middle

   

In [12]:
def get_difference( A , B):

    dist = np.zeros(( A.shape[0],1))
    for i in range(A.shape[1]):
        dist[i] = np.linalg.norm( A[i,:] - B[i,:])

    return dist

In [13]:
def kd_matching(s_k , Np , mesh_points , Ntriangles , mesh_vertices ):
    
    rectangles = get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices)

    kdtree = KD_tree(Np , mesh_points , Ntriangles , mesh_vertices , rectangles)
    
    Ns = s_k.shape[0]
    
    closest_p = []
    for i in range(Ns):
        min_dist = np.finfo(np.float32).max
        tmp = kdtree.FindClosestPoint( s_k[i] )
        closest_p.append(tmp)
    return closest_p

In [14]:
data_dir = 'DATA/'
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K']
runtype = ['Debug' ,'Debug' ,'Debug' ,'Debug' ,'Debug' ,'Debug' ,'Unknown','Unknown','Unknown','Unknown']
pa = "PA5"
filename = '/Problem5MeshFile.sur'

In [15]:
def get_models(filename):
    
    models_f = glob.glob( filename )
    models_file = open( models_f[0] , "r")
    models_lines = models_file.read().splitlines()
    models_data = []
    
    first_line = models_lines[0].split()
    Np = int((first_line[1].split('='))[1])
    N_models = int( (first_line[2].split('='))[1])
    #print(Np)
    #print(N_models)
    for num in range(1,len(models_lines)):
        if(num%(Np+1) == 1):
            continue
        models_data.append( models_lines[num].split(',') )
        for i in range(len( models_data[-1])):
            models_data[-1][i] = models_data[-1][i].strip() # to remove space and tabs

    models = []
    for i in range(N_models+1):        
        model = np.asarray( models_data[ i*Np : i*Np + Np ] ).astype( float )
        models.append( model )
    models = np.array(models).astype(float)

    return models

In [16]:
def ICP(d_k , Np , mesh_points , Ntriangles , mesh_vertices ,brute_force = False ):
    
    F_reg = np.eye(4)
    
    rectangles = get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices)
    if(brute_force == False):
        kdtree = KD_tree(Np , mesh_points , Ntriangles , mesh_vertices , rectangles)
    
    Ns = d_k.shape[0]
    
    closest_p = []
    iteration = 40
    c_k = []
    old_pts = []
    
    closest_p = []

    
    
    for i in range(iteration):
        s_k_i = points_transform(F_reg , d_k) 
        #c_k = icpm.ICPmatch(s_i, vCoords, vIndices, tree=tree, oldpts=old_pts, usetree=True)
        closest_p = []
        if(brute_force):
            for j in range(Ns):
                min_dist = np.finfo(np.float32).max
                P_closest = []
                for k in range(Ntriangles):

                    tmp = point_to_triangle(s_k_i[j] , k )

                    dist = np.linalg.norm(s_k_i[j] - tmp)
                    if min_dist > dist:
                        min_dist = dist
                        P_closest = tmp
                closest_p.append(P_closest)
       
            
        if(brute_force==False):
            for j in range(Ns):
                min_dist = np.finfo(np.float32).max
                tmp = kdtree.FindClosestPoint( s_k_i[j] )
                closest_p.append(tmp)

            
        c_k = np.array(closest_p)
        # Update guess
        old_pts = c_k

        deltaF_reg = registration_3d( s_k_i , c_k)

        F_regNew = deltaF_reg @ F_reg

        if np.sum(np.abs(F_reg - F_regNew)) < 1e-4:
            return c_k , F_regNew

        F_reg = F_regNew


            
    return c_k , F_reg

In [17]:
def get_bycentrix( point ,p,q,r ):
    
    s1 = np.cross( ( r - p ) , ( q - p ) )
    s1 = np.linalg.norm(s1)
    
    s2 = np.cross( ( r - point ) , ( q - point ) )
    s2 = np.linalg.norm(s2)
    
    s3 = np.cross( ( p - point ) , ( q - point ) )
    s3 = np.linalg.norm(s3)
    
    a = s2 / s1
    c = s3 / s1
    b = 1 - a - c
    #print("case:")
    #print(a*p  + b*q + c*r - point)
    return np.asarray([a,b,c]).astype(float)

In [18]:
def deformable_ICP(d_k, Np, mesh_points, Ntriangles, mesh_vertices, models, brute_force = False ):
    
    rectangles = get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices)
    if(brute_force == False):
        kdtree = KD_tree(Np , mesh_points , Ntriangles , mesh_vertices , rectangles)
    
    Ns = d_k.shape[0]
    
    closest_p = []
    iteration = 40
    c_k = []
    old_pts = []
    closest_p = []
    
    Lambda = np.ones( (6,1) )
    Lambda_old = np.ones( (6,1) )*1000
    
    F_reg = np.eye(4)
    F_reg_old = np.eye(4)*1000
    
    while(True):
        if( np.sum(np.abs(F_reg - F_reg_old)) < 1e-4 ):
            break
        if(iteration < 0):
            break
        iteration = iteration - 1
        #print("11111111111111111")
        Lambda_old = np.ones( (6,1) )
        Lambda_old = Lambda_old * 1000.0
        
        F_reg_old = F_reg
        
        s_k = points_transform(F_reg , d_k) 
        
        q0 = np.zeros( (s_k.shape[0] , s_k.shape[1] ) )
        
        # n*3*6 easier for least square
        qm = np.zeros( (  s_k.shape[0],s_k.shape[1] , models.shape[0]-1) )
        #print("qm",qm.shape)
        while( np.linalg.norm(Lambda - Lambda_old ) > 1e-4 ):
            
            Lambda_old = Lambda
            
            for i in range( d_k.shape[0] ):
                point , triangle_index = kdtree.FindClosestPoint( s_k[i] )
                #print("closest point")
                #print(point)
                #print(point_to_triangle(s_k[i], triangle_index))
                closest_p.append(point)
                
                p = mesh_points[mesh_vertices[ triangle_index,0]] 
                q = mesh_points[mesh_vertices[ triangle_index,1]]
                r = mesh_points[mesh_vertices[ triangle_index,2]]
                
                ################ to be continued
                bary = get_bycentrix(point, p, q, r)
                #print("triangle_index ",triangle_index)
                #print(mesh_vertices.shape)
                q0_i = 0
                for j in range(3):
                    q0[i] += models[0][mesh_vertices[ triangle_index,j]]*bary[j] 
                
                for j in range(1,models.shape[0]):
                    #print(j)
                    #print(models.shape)
                    for k in range(3):
                        #print("test")
                        #print(qm[i ,: ,j-1].shape)
                        #print( (models[j][mesh_vertices[ triangle_index,k]]*bary[k]).shape)
                        
                        qm[i ,: ,j-1] += models[j][mesh_vertices[ triangle_index,k]]*bary[k] 
                        #print(qm[i ,: ,j-1] )
                #print("q0_i ",q0_i)
                #254
                #Lambda = least_squares(s,q0,qm,size,bone_mode.num_m);


            
            # s_k = q0 + lambda * qm 
            A = np.zeros( (s_k.shape[0]*s_k.shape[1] , models.shape[0]-1 ) )
            B = np.zeros( s_k.shape[0]*s_k.shape[1] )

            for i in range(s_k.shape[0]):
                A[i*3] = qm[i,0,:] 
                A[i*3+1] = qm[i,1,:]
                A[i*3+2] = qm[i,2,:]
                
                B[i*3] = s_k[i,0] - q0[i,0] 
                B[i*3+1] = s_k[i,1] - q0[i,1]
                B[i*3+2] = s_k[i,2] - q0[i,2]
                

            Lambda = np.linalg.lstsq(A, B, rcond=None)[0]
            if(False):
                A = qm
                B = s_k - q0
                print(A.shape)
                print(B.shape)
                print(np.abs( (A@Lambda) - B ))

            # use Lambda to update KDtree
    return

In [19]:
if (True):
    num=0
#for num in range(1):
    data_type = num
    letter = num
    Np,mesh_points,Ntriangles,mesh_vertices = read_mesh(data_dir , '/Problem5MeshFile.sur')
    N_Ap,A_points,A_tip = read_body(data_dir , '/Problem5-BodyA.txt')
    N_Bp,B_points,B_tip = read_body(data_dir , '/Problem5-BodyB.txt')
    print(mesh_vertices.shape)
    sample_dir = '/PA5-' + letters[num] + '-' + runtype[num] + '-SampleReadingsTest.txt'
    A_frames , B_frames = read_sample(data_dir, sample_dir, N_Ap , N_Bp)
    
    d_k = get_d_k(A_frames, A_points, A_tip, B_frames, B_points, B_tip)
    F_reg = np.eye(4)
    #print("d_k points : ",d_k.shape[0])
    
    models = get_models(data_dir + 'Problem5Modes.txt')
    
    deformable_ICP(d_k , Np , mesh_points , Ntriangles , mesh_vertices ,models ,brute_force = False       )
if False:
    s_k = points_transform(F_reg , d_k) 
    
    difference = get_difference(s_k , c_k)
    print( '/PA5-' + letters[num] + '-' + runtype[num])
    print(np.sum(difference))
    for i in range(s_k.shape[0]):
        print( np.round(s_k[i],2), ' ' ,np.round(c_k[i],2) )

(3135, 6)
(150, 3, 6)
(150, 3)
[[2.59366871 0.0921621  0.43563565]
 [0.8950859  1.09395178 0.01786325]
 [0.26788727 1.14130561 0.2427477 ]
 [1.29490879 0.24608759 0.91557871]
 [1.83745301 1.02244813 0.42055842]
 [1.93486695 1.36297299 0.15347301]
 [0.01319095 0.02913198 0.1880687 ]
 [2.33099689 1.67540973 0.79946204]
 [1.30567761 0.23654505 3.94912037]
 [0.80979262 1.93874716 1.40624324]
 [2.2894985  0.84095539 2.04248631]
 [2.86924279 0.38236038 0.87631851]
 [0.69509142 3.21084741 0.15548233]
 [1.07338128 0.00954251 2.14498821]
 [3.06995793 1.08896992 0.61765601]
 [1.35323697 2.07395079 0.36666432]
 [0.14835904 2.01873693 1.13970748]
 [0.85921036 1.34244641 2.21850326]
 [0.37706739 1.95657545 0.61358939]
 [2.11466677 1.99264281 0.85706075]
 [1.33333257 0.66862078 0.96879652]
 [0.12046077 1.98426117 1.29741759]
 [1.17029064 0.66502526 2.04927713]
 [1.64564873 2.18838163 1.1450694 ]
 [0.50586656 0.28657526 0.48274301]
 [1.15583627 0.95650545 1.10283032]
 [1.02481474 3.03951374 0.0703827

(150, 3, 6)
(150, 3)
[[1.31890630e+01 3.08701810e+01 8.98263647e+01]
 [3.80165831e+01 2.90799351e+01 2.41755189e+01]
 [7.58608259e+00 5.20652776e+01 7.89466287e+00]
 [2.34705079e+00 6.57087302e+01 4.16523410e+01]
 [4.42109532e+01 4.89411928e+00 5.02613776e+01]
 [1.74403157e+01 2.10938894e+01 6.78597118e+01]
 [2.20201846e-01 3.00369777e+01 6.28043883e+01]
 [5.00677651e+01 1.94694752e+01 5.76709479e+01]
 [3.19620896e+01 3.50293689e+01 5.68224659e+01]
 [3.87560976e+01 6.90026187e+01 2.13079838e+01]
 [6.85999335e+00 1.05675983e+00 4.78852942e+01]
 [5.91972006e+01 3.46541925e+00 5.27901399e+01]
 [2.63958721e+01 2.85942221e+01 5.86271147e+01]
 [2.42753454e+01 2.47850170e+01 5.67736600e+01]
 [2.67181351e+01 8.67802955e+00 2.79836508e+01]
 [3.34568971e+01 6.43358959e+01 5.54467950e+01]
 [5.98025455e+01 1.75849577e+01 2.29141066e+01]
 [2.74697250e+00 6.40238683e+01 3.50598975e+01]
 [4.08612607e+01 1.92489302e+01 1.45940247e+01]
 [4.95636180e+01 1.13987421e+00 6.12208102e+01]
 [1.80797457e+01 6.

(150, 3, 6)
(150, 3)
[[2.89717948e+01 6.16481999e+01 1.79217094e+02]
 [7.51380803e+01 5.92538220e+01 4.83689010e+01]
 [1.49042779e+01 1.02989250e+02 1.60320734e+01]
 [3.39919279e+00 1.31663548e+02 8.23891032e+01]
 [9.02593595e+01 8.76579043e+00 1.00943314e+02]
 [3.68154983e+01 4.35507517e+01 1.35872897e+02]
 [4.27212743e-01 6.01030873e+01 1.25420708e+02]
 [1.02466527e+02 4.06143602e+01 1.16141358e+02]
 [6.52298568e+01 7.02952828e+01 1.17594052e+02]
 [7.67024026e+01 1.36066490e+02 4.40222109e+01]
 [1.60094852e+01 1.27256427e+00 9.37281021e+01]
 [1.15525158e+02 6.54847812e+00 1.06456598e+02]
 [5.34868356e+01 5.39775968e+01 1.17409712e+02]
 [4.74773095e+01 4.95604914e+01 1.15692308e+02]
 [5.65062282e+01 1.62670892e+01 5.65849576e+01]
 [6.82670312e+01 1.26597841e+02 1.10526926e+02]
 [1.19753450e+02 3.31511784e+01 4.69679207e+01]
 [4.63473464e+00 1.29390183e+02 6.79012917e+01]
 [8.20995888e+01 4.04544358e+01 2.85744601e+01]
 [1.01241903e+02 4.27239123e+00 1.23298681e+02]
 [3.74928241e+01 1.

(150, 3, 6)
(150, 3)
[[4.47545266e+01 9.24262188e+01 2.68607823e+02]
 [1.12259578e+02 8.94277089e+01 7.25622831e+01]
 [2.22224732e+01 1.53913222e+02 2.41694840e+01]
 [4.45133480e+00 1.97618366e+02 1.23125865e+02]
 [1.36307766e+02 1.26374616e+01 1.51625250e+02]
 [5.61906810e+01 6.60076141e+01 2.03886081e+02]
 [6.34223640e-01 9.01691970e+01 1.88037028e+02]
 [1.54865289e+02 6.17592451e+01 1.74611768e+02]
 [9.84976240e+01 1.05561197e+02 1.78365639e+02]
 [1.14648708e+02 2.03130362e+02 6.67364380e+01]
 [2.51589770e+01 1.48836872e+00 1.39570910e+02]
 [1.71853116e+02 9.63153698e+00 1.60123057e+02]
 [8.05777991e+01 7.93609715e+01 1.76192309e+02]
 [7.06792736e+01 7.43359659e+01 1.74610956e+02]
 [8.62943212e+01 2.38561488e+01 8.51862644e+01]
 [1.03077165e+02 1.88859786e+02 1.65607056e+02]
 [1.79704355e+02 4.87173991e+01 7.10217348e+01]
 [6.52249678e+00 1.94756498e+02 1.00742686e+02]
 [1.23337917e+02 6.16599414e+01 4.25548954e+01]
 [1.52920188e+02 7.40490825e+00 1.85376552e+02]
 [5.69059024e+01 1.

(150, 3, 6)
(150, 3)
[[6.05372583e+01 1.23204238e+02 3.57998552e+02]
 [1.49381075e+02 1.19601596e+02 9.67556652e+01]
 [2.95406686e+01 2.04837194e+02 3.23068946e+01]
 [5.50347680e+00 2.63573184e+02 1.63862628e+02]
 [1.82356172e+02 1.65091327e+01 2.02307186e+02]
 [7.55658636e+01 8.84644765e+01 2.71899266e+02]
 [8.41234537e-01 1.20235307e+02 2.50653347e+02]
 [2.07264051e+02 8.29041301e+01 2.33082178e+02]
 [1.31765391e+02 1.40827111e+02 2.39137225e+02]
 [1.52595013e+02 2.70194233e+02 8.94506650e+01]
 [3.43084689e+01 1.70417316e+00 1.85413718e+02]
 [2.28181074e+02 1.27145958e+01 2.13789515e+02]
 [1.07668763e+02 1.04744346e+02 2.34974906e+02]
 [9.38812377e+01 9.91114404e+01 2.33529605e+02]
 [1.16082414e+02 3.14452084e+01 1.13787571e+02]
 [1.37887299e+02 2.51121731e+02 2.20687187e+02]
 [2.39655259e+02 6.42836199e+01 9.50755488e+01]
 [8.41025892e+00 2.60122813e+02 1.33584080e+02]
 [1.64576245e+02 8.28654470e+01 5.65353308e+01]
 [2.04598472e+02 1.05374253e+01 2.47454423e+02]
 [7.63189807e+01 2.

...

(150, 3, 6)
(150, 3)
[[139.45091709 277.09433224 804.95219703]
 [334.98856085 270.47103014 217.72257567]
 [ 66.13164518 459.45705385  72.99394737]
 [ 10.76418681 593.34727276 367.54643902]
 [412.59820314  35.86748847 455.716866  ]
 [172.44177677 200.74878835 611.96519011]
 [  1.87628902 270.56585481 563.73494524]
 [469.2578611  188.62855485 525.43422718]
 [298.10422737 317.15668036 542.99515633]
 [342.32653739 605.5135908  203.0218004 ]
 [ 80.05592811   2.78319538 414.62775722]
 [509.8208627   28.12989018 482.12180759]
 [243.1235802  231.66121973 528.88789079]
 [209.89105815 222.98881272 528.12284579]
 [265.02287958  69.39050661 256.79410525]
 [311.93796984 562.43145634 496.08784036]
 [539.4097822  142.11472353 215.34461922]
 [ 17.84906962 586.95438625 297.79105133]
 [370.76788557 188.89297499 126.43750752]
 [462.98989638  26.20001038 557.84377819]
 [173.38437225 565.00179623 562.47955259]
 [383.56361384 299.32763441 406.33594064]
 [525.439709    72.32215152 618.78264197]
 [172.6206231

(150, 3, 6)
(150, 3)
[[155.23364885 307.87235114 894.34292607]
 [372.11005807 300.64491702 241.91595777]
 [ 73.44984051 510.38102588  81.13135794]
 [ 11.81632882 659.30209058 408.28320128]
 [458.64660938  39.73915962 506.39880205]
 [191.81695941 223.20565072 679.9783749 ]
 [  2.08329992 300.63196445 626.35126485]
 [521.6566231  209.77343981 583.90463709]
 [331.37199459 352.4225943  603.76674263]
 [380.27284236 672.57746231 225.73602747]
 [ 89.20541996   2.99899982 460.4705651 ]
 [566.14882047  31.21294905 535.78826605]
 [270.21454372 257.04459443 587.6704878 ]
 [233.09302225 247.76428718 587.04149401]
 [294.81097264  76.97956624 285.39541206]
 [346.74810393 624.6934014  551.16797103]
 [599.36068679 157.68094427 239.3984333 ]
 [ 19.73683176 652.32070099 330.63244556]
 [412.00621368 210.0984806  140.41794287]
 [514.66818118  29.3325274  619.92164919]
 [192.79745056 627.70548239 624.8696366 ]
 [426.16840863 332.36578699 451.628536  ]
 [583.6918666   80.28405444 687.76396631]
 [191.9835422

(150, 3, 6)
(150, 3)
[[ 171.01638061  338.65037005  983.73365512]
 [ 409.23155529  330.8188039   266.10933987]
 [  80.76803583  561.30499791   89.2687685 ]
 [  12.86847082  725.25690839  449.01996354]
 [ 504.69501562   43.61083077  557.08073809]
 [ 211.19214204  245.6625131   747.99155969]
 [   2.29031082  330.69807409  688.96758447]
 [ 574.0553851   230.91832476  642.375047  ]
 [ 364.63976181  387.68850823  664.53832893]
 [ 418.21914734  739.64133383  248.45025455]
 [  98.3549118     3.21480427  506.31337298]
 [ 622.47677824   34.29600792  589.4547245 ]
 [ 297.30550723  282.42796913  646.45308482]
 [ 256.29498635  272.53976165  645.96014223]
 [ 324.59906569   84.56862588  313.99671886]
 [ 381.55823803  686.95534646  606.2481017 ]
 [ 659.31159137  173.247165    263.45224738]
 [  21.6245939   717.68701573  363.47383979]
 [ 453.24454178  231.3039862   154.39837821]
 [ 566.34646597   32.46504442  681.99952018]
 [ 212.21052887  690.40916855  687.25972061]
 [ 468.77320341  365.40393957  496

(150, 3, 6)
(150, 3)
[[ 186.79911236  369.42838895 1073.12438416]
 [ 446.3530525   360.99269078  290.30272197]
 [  88.08623115  612.22896994   97.40617906]
 [  13.92061282  791.21172621  489.75672579]
 [ 550.74342186   47.48250192  607.76267414]
 [ 230.56732468  268.11937547  816.00474449]
 [   2.49732172  360.76418374  751.58390408]
 [ 626.4541471   252.06320971  700.84545691]
 [ 397.90752903  422.95442217  725.30991523]
 [ 456.16545231  806.70520534  271.16448162]
 [ 107.50440365    3.43060871  552.15618086]
 [ 678.80473601   37.37906678  643.12118296]
 [ 324.39647075  307.81134384  705.23568183]
 [ 279.49695044  297.31523612  704.87879045]
 [ 354.38715875   92.15768551  342.59802567]
 [ 416.36837212  749.21729152  661.32823237]
 [ 719.26249595  188.81338573  287.50606146]
 [  23.51235604  783.05333047  396.31523402]
 [ 494.48286989  252.50949181  168.37881356]
 [ 618.02475077   35.59756144  744.07739118]
 [ 231.62360718  753.11285471  749.64980461]
 [ 511.3779982   398.44209216  542

KeyboardInterrupt: 

In [ ]:
models.shape

In [ ]:
d_k.shape

In [ ]:
N_Ap

In [ ]:
A_frames.shape